<a href="https://colab.research.google.com/github/emilbiju/quora-qpairs/blob/Pavan-Vemuri/GloVe_100d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pylab import *
import pandas as pd
import string

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

from google.colab import drive
drive.mount('/content/drive')
prefix = '/content/drive/My Drive/EE5180/Project/'

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
#Creating dictionary of embeddings
embeddings_index = {}
f = open(prefix+'glove.6B.100d.txt')
for line in f:
    values = line.split(' ')
    word = values[0] ## The first entry is the word
    coefs = np.asarray(values[1:], dtype='float32') ## These are the vecotrs representing the embedding for the word
    embeddings_index[word] = coefs
f.close()

#Raw Data
data = pd.read_csv(prefix + "questions.csv")
data = data.drop(['id','qid1','qid2'],axis=1)
dataArray = column_stack((data.question1,data.question2,data.is_duplicate))
data.head()

,question1,question2,is_duplicate
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [3]:
test_data, gg = train_test_split(dataArray, test_size= 0.9)
rest, train_data = train_test_split(gg, test_size= 0.5)
print(len(test_data),len(train_data))

40435 181958


# 100d GloVe Embeddings

In [4]:
def preProcess(doc):
  processed = []; uk=[[],[]]
  for i in range(2):
    doc_processed = str(doc[i]).lower()
    translator = str.maketrans('','',(string.punctuation))
    doc_processed = doc_processed.translate(translator)
    doc_processed = word_tokenize(doc_processed)
    vect = zeros(100)
    for word in doc_processed:
      if word in embeddings_index:
        vect = vect + embeddings_index[word]
      else:
        uk[i].append(word)
    processed.append(vect)
  out = abs(processed[0]-processed[1])
  i = len(set(uk[0])-set(uk[1])) + len(set(uk[1])-set(uk[0]))
  out = out + i*ones(100)
  return out

#Preprocessiong
vectorData_train = []; labels_train = []
for w in train_data:
  vectorData_train.append(preProcess(w)) 
  labels_train.append(w[2])
vectorData_train = array(vectorData_train)
labels_train = array(labels_train)
labels_train = labels_train*2 - ones(len(labels_train))

vectorData_test = []; labels_test = []
for w in test_data:
  vectorData_test.append(preProcess(w)) 
  labels_test.append(w[2])
vectorData_test = array(vectorData_test)
labels_test = array(labels_test)
labels_test = labels_test*2 - ones(len(labels_test))

In [6]:
#Logistic Regression
reg = LogisticRegression(max_iter=1000).fit(vectorData_train, labels_train)
train = reg.score(vectorData_train, labels_train)
test = reg.score(vectorData_test, labels_test)

print( 'Train accuracy: '+ str(100*train) + '%')
print( 'Test accuracy: '+ str(100*test) + '%')

Train accuracy: 67.35015772870663%
Test accuracy: 66.89748979844194%


In [7]:
#Polynomial logistic Regression
def transform(data,T):
  out = data
  for i in range(2,T+1):
    out = concatenate((out, data**i), axis=1)
  return out
T=2
reg = LogisticRegression(random_state=0,max_iter=10000).fit(transform(vectorData_train,T), labels_train)
train = reg.score(transform(vectorData_train,T), labels_train)
test = reg.score(transform(vectorData_test,T), labels_test)

print( 'Train accuracy: '+ str(100*train) + '%')
print( 'Test accuracy: '+ str(100*test) + '%')

Train accuracy: 68.6422141373284%
Test accuracy: 68.09199950537901%


In [9]:
#Random forest
reg = RandomForestClassifier(max_depth = 20,random_state=0).fit(vectorData_train, labels_train)
train = reg.score(vectorData_train, labels_train)
test = reg.score(vectorData_test, labels_test)

print( 'Train accuracy: '+ str(100*train) + '%')
print( 'Test accuracy: '+ str(100*test) + '%')

Train accuracy: 97.61593334725596%
Test accuracy: 69.99381723754173%


# Bad Results

In [8]:
#Adaboost
reg = AdaBoostClassifier(n_estimators=10, random_state=0).fit(vectorData_train, labels_train)
train = reg.score(vectorData_train, labels_train)
test = reg.score(vectorData_test, labels_test)

print( 'Train accuracy: '+ str(100*train) + '%')
print( 'Test accuracy: '+ str(100*test) + '%')

Train accuracy: 66.47303223820882%
Test accuracy: 65.79943118585383%
